In [88]:
import pandas as pd, numpy
import csv
import requests
import os
import json
import datetime
import dateutil.parser
import unicodedata
import time
import re
import nltk

In [89]:
#raw data
all_china = pd.read_csv('Desktop/w266-final-project/all_china_full_nort.csv')
pro_china = pd.read_csv('Desktop/w266-final-project/pro_china_full_nort.csv')

In [90]:
#Text preprocessing
# source: https://medium.com/analytics-vidhya/text-classification-using-word-embeddings-and-deep-learning-in-python-classifying-tweets-from-6fe644fcfc81
def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
        stop_words = nltk.corpus.stopwords.words("english")) -> str:
    """
    A method to clean text 
    """
    # Clean urls
    string = re.sub('@[^\s]+','',string)
    string = re.sub('#[^\s]+','',string)

    string = re.sub(r'https?://\S+|www\.\S+', '', string)
    string = re.sub(r'<.*?>', '', string)
    
    #remove random char... non-letters
    string = re.sub(r'[^a-zA-Z0-9 -]','', string)
    

    
    # Remove words containing numbers 
    string = re.sub("\S*\d\S*", '', string).strip()
    
    # Remove the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()
    
    string = re.sub(r'rt ','', string)

    return string   

In [91]:
pro_china['text'] = [clean_text(x) for x in pro_china['text']]
all_china['text'] = [clean_text(x) for x in all_china['text']]


In [92]:
#3 sets of tweets 
is_ccp = pro_china.loc[pro_china['is_ccp']==1]['text']
pro_china = pro_china.loc[pro_china['is_ccp']==0]['text']
all_china = all_china.loc[all_china['is_ccp']==0]['text']




In [66]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

df_list = [is_ccp, pro_china, all_china]

def filter_tweets(df, query):
    # takes in a list of tweets and then returns a list of new stuff i guess 
    return df[df.str.contains(query, case=False)]

def sentiment_ratio(tweets):
    positive = 0
    negative = 0
    for tweet in tweets:
        if sia.polarity_scores(tweet)["compound"] > 0:
            positive += 1 
        else:
            negative +=1 
    print(len(tweets), " tweets")
    return [negative / len(tweets), positive / len(tweets)]

In [67]:
def get_sentiment_ratio_for_query(query, input_list):
    lst = [sentiment_ratio(filter_tweets(i, query)) for i in input_list]
    print("Query:", query)
    print("is_ccp:", lst[0])
    print("pro_china:", lst[1])
    print("all_china:", lst[2])


results= get_sentiment_ratio_for_query("China", df_list)
results

5012  tweets
12258  tweets
6579  tweets
Query: China
is_ccp: [0.6516360734237829, 0.34836392657621706]
pro_china: [0.579295154185022, 0.42070484581497797]
all_china: [0.6438668490652075, 0.35613315093479253]


In [99]:
get_sentiment_ratio_for_query("taiwan", df_list)

70  tweets
255  tweets
285  tweets
Query: taiwan
is_ccp: [0.7142857142857143, 0.2857142857142857]
pro_china: [0.5764705882352941, 0.4235294117647059]
all_china: [0.6385964912280702, 0.36140350877192984]


In [68]:
get_sentiment_ratio_for_query("xi jinping", df_list)

46  tweets
278  tweets
66  tweets
Query: xi jinping
is_ccp: [0.8043478260869565, 0.1956521739130435]
pro_china: [0.46402877697841727, 0.5359712230215827]
all_china: [0.7272727272727273, 0.2727272727272727]


In [69]:
get_sentiment_ratio_for_query("taiwan", df_list)

70  tweets
255  tweets
285  tweets
Query: taiwan
is_ccp: [0.7142857142857143, 0.2857142857142857]
pro_china: [0.5764705882352941, 0.4235294117647059]
all_china: [0.6385964912280702, 0.36140350877192984]


In [70]:
get_sentiment_ratio_for_query("xinjiang", df_list)

5610  tweets
1024  tweets
3287  tweets
Query: xinjiang
is_ccp: [0.5775401069518716, 0.42245989304812837]
pro_china: [0.646484375, 0.353515625]
all_china: [0.6924247033769395, 0.30757529662306055]


In [71]:
get_sentiment_ratio_for_query("uyghur", df_list)

192  tweets
410  tweets
608  tweets
Query: uyghur
is_ccp: [0.3385416666666667, 0.6614583333333334]
pro_china: [0.6926829268292682, 0.3073170731707317]
all_china: [0.7154605263157895, 0.2845394736842105]


In [77]:
get_sentiment_ratio_for_query("uyghur", df_list)

192  tweets
410  tweets
608  tweets
Query: uyghur
is_ccp: [0.3385416666666667, 0.6614583333333334]
pro_china: [0.6926829268292682, 0.3073170731707317]
all_china: [0.7154605263157895, 0.2845394736842105]


In [78]:
get_sentiment_ratio_for_query("china", df_list)

5012  tweets
12258  tweets
6579  tweets
Query: china
is_ccp: [0.6516360734237829, 0.34836392657621706]
pro_china: [0.579295154185022, 0.42070484581497797]
all_china: [0.6438668490652075, 0.35613315093479253]


In [82]:
get_sentiment_ratio_for_query("surveillance", df_list)

1  tweets
43  tweets
49  tweets
Query: surveillance
is_ccp: [1.0, 0.0]
pro_china: [0.6511627906976745, 0.3488372093023256]
all_china: [0.4489795918367347, 0.5510204081632653]


In [75]:
get_sentiment_ratio_for_query("russia", df_list)

26  tweets
704  tweets
404  tweets
Query: russia
is_ccp: [0.34615384615384615, 0.6538461538461539]
pro_china: [0.6193181818181818, 0.3806818181818182]
all_china: [0.6089108910891089, 0.3910891089108911]


In [96]:
get_sentiment_ratio_for_query("hong kong", df_list)

1999  tweets
822  tweets
3888  tweets
Query: hong kong
is_ccp: [0.6913456728364182, 0.30865432716358177]
pro_china: [0.5924574209245742, 0.4075425790754258]
all_china: [0.6072530864197531, 0.39274691358024694]


In [98]:
get_sentiment_ratio_for_query("wengui", df_list)

1438  tweets
4  tweets
3520  tweets
Query: wengui
is_ccp: [0.6981919332406119, 0.30180806675938804]
pro_china: [0.5, 0.5]
all_china: [0.6991477272727272, 0.3008522727272727]


In [86]:
[print(i) for i in [filter_tweets(i, "wengui") for i in df_list][1]]

cambridge analytica bannon mastermind stuff definitely played areas specifically hunter biden laptop op antichina far right talking point engine largely mastermind also worth looking funder guo wengui
website gnews first leak data hunter laptop appendage gtv coowned steve bannon guo wengui
last week american rightwing political agitator steve bannon joined tweeting exiled businessman guo wengui former soccer star hao haidong make video calling downfall chinese communist party rather bizarre
clear case influencing us election collusion foreign actors hong kongs apple daily fugitive chinese billionaire guo wengui maybe time arrest


[None, None, None, None]

In [290]:
test = pd.read_csv('Downloads/test.csv')
test = test.loc[test['is_ccp']==0]['text']


In [312]:
[print(i) for i in [filter_tweets(i, "wengui") for i in [test]]]

19328    RT @WendySiegelman: @JamieMCorey @RepublicanAG...
19330    5 potential solutions for maintaining attorney...
19340    RT @Stephan59317911: Stephen K. Bannon and Wen...
19353    Guo Wengui is a hitting dog https://t.co/FKSI5...
19364    RT @karolcummins: @DemopJ Mr. Blair &amp; Stev...
                               ...                        
38599    @MirqIFHbgcyBzLJ Guo Wengui and Bannon only wa...
38613    RT @Tiger789: 郭骗(aka Guo wengui,Miles Kwok,Kwo...
38619              @SweetZ55865791 Guo Wengui is a pet dog
38621    @mamnoon @JDiamond1 My understanding is that t...
38637    RT @ScottMStedman: Steve Bannon is under feder...
Name: text, Length: 3164, dtype: object


[None]

In [308]:
[print(i) for i in [filter_tweets(i, "hong kong") for i in [all_china]]]

19329    mlb boycotts georgia signs new deal chinese fi...
19330    tomorrow hong kongthe national championship vs...
19331    hong kong polices deputy commissioner oscar kw...
19335    mlb boycotts georgia signs new deal chinese fi...
19344    hong kong long trumpeted one transparent regis...
                               ...                        
38626    musical easter gift microsoft asiapacific work...
38637    mlb boycotts georgia signs new deal chinese fi...
38640    convictions hong kongs ardent protectors right...
38641    mlb boycotts georgia signs new deal chinese fi...
38642                                 brave hong kong soul
Name: text, Length: 3475, dtype: object


[None]

In [324]:
get_sentiment_ratio_for_query("covid", df_list)

8  tweets
302  tweets
204  tweets
Query: covid
is_ccp: [0.875, 0.125]
pro_china: [0.6986754966887417, 0.30132450331125826]
all_china: [0.8921568627450981, 0.10784313725490197]


In [57]:
is_ccp_samples = pd.DataFrame(is_ccp.sample(1000))
is_ccp_samples['label'] = "is_ccp"
pro_china_samples = pd.DataFrame(pro_china.sample(1000))
pro_china_samples['label'] = "pro_china"
all_china_samples = pd.DataFrame(all_china.sample(1000))
all_china_samples['label'] = "all_china"

,text,label
23584,@_Expanded_Mind_ @ErwinBoydens @GretaThunberg ...,all_china
34095,When our companies sourced labor in China it u...,all_china
21802,It’s like the world following china on Muslims,all_china
24349,@BooreamFrancois And one of the highest % of e...,all_china
31218,@MeganMargKing @Kollerkunzultz I am ashamed to...,all_china
...,...,...
31056,I remember seeing this years ago...\nStill a s...,all_china
27671,@JMichaelWaller Guo wengui is a disgusting per...,all_china
31270,@evander_87 @ggreenwald He took millions of do...,all_china
21524,@Lonelystarrd22 @The_Acumen @gol_mia Not in Ja...,all_china


In [60]:
all_samples = pd.concat([is_ccp_samples,pro_china_samples,all_china_samples])

In [61]:
all_samples.to_csv('Desktop/w266-final-project/classify_samples_nort.csv')